In [1]:
import shutil
import os

# Define source dataset folder (where the current dataset is stored)
source_folder = r"C:/Users/abina/Downloads/archive"  # Modify this to your dataset location

# Define destination folder correctly
destination_folder = r"C:/Users/abina/fish_recognition/dataset"

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

In [5]:
import os
source_folder = r"C:/Users/abina/Downloads/archive/FishImgDataset"
print("Folders found:", os.listdir(source_folder))


Folders found: ['test', 'train', 'val']


In [9]:

import shutil
import os

# Define paths
source_folder = r"C:/Users/abina/Downloads/archive"
destination_folder = r"C:/Users/abina/fish_recognition/dataset"

# Ensure destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Get all folders inside the source dataset
folders = os.listdir(source_folder)

print(f"📂 Source Folder Contents: {folders}")  # Debugging step

# Move each folder into the project's dataset
for folder in folders:
    source_path = os.path.join(source_folder, folder)
    destination_path = os.path.join(destination_folder, folder)

    # Ensure it's a valid folder before moving
    if os.path.isdir(source_path):
        print(f"🔄 Moving {source_path} → {destination_path}")  # Debugging step
        shutil.move(source_path, destination_path)
    else:
        print(f"⚠️ Skipped (Not a folder): {source_path}")  # Debugging step

# Final verification
print("📂 Folders in Project Dataset:", os.listdir(destination_folder))


📂 Source Folder Contents: ['FishImgDataset']
🔄 Moving C:/Users/abina/Downloads/archive\FishImgDataset → C:/Users/abina/fish_recognition/dataset\FishImgDataset
📂 Folders in Project Dataset: ['FishImgDataset']


In [11]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os
import numpy as np

# Load ResNet-18 for feature extraction
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()  # Remove the classification layer to get feature embeddings
resnet18.eval()  # Set model to evaluation mode

# Define dataset paths using absolute paths
dataset_path = r"C:/Users/abina/fish_recognition/dataset/FishImgDataset"
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "val")

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

# Load dataset
train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Debugging print statements to verify dataset loading
print(f"✅ Training directory: {train_dir}")
print(f"✅ Validation directory: {val_dir}")
print(f"📂 Total Classes: {len(train_data.classes)}")
print("📂 Class Names:", train_data.classes)

C:\Users\abina\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abina\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Training directory: C:/Users/abina/fish_recognition/dataset/FishImgDataset\train
✅ Validation directory: C:/Users/abina/fish_recognition/dataset/FishImgDataset\val
📂 Total Classes: 31
📂 Class Names: ['Bangus', 'Big Head Carp', 'Black Spotted Barb', 'Catfish', 'Climbing Perch', 'Fourfinger Threadfin', 'Freshwater Eel', 'Glass Perchlet', 'Goby', 'Gold Fish', 'Gourami', 'Grass Carp', 'Green Spotted Puffer', 'Indian Carp', 'Indo-Pacific Tarpon', 'Jaguar Gapote', 'Janitor Fish', 'Knifefish', 'Long-Snouted Pipefish', 'Mosquito Fish', 'Mudfish', 'Mullet', 'Pangasius', 'Perch', 'Scat Fish', 'Silver Barb', 'Silver Carp', 'Silver Perch', 'Snakehead', 'Tenpounder', 'Tilapia']


In [13]:
def extract_features(model, loader):
    features = []
    labels = []
    
    with torch.no_grad():
        for images, targets in loader:
            outputs = model(images)  
            features.append(outputs.view(outputs.shape[0], -1).numpy())  
            labels.extend(targets.numpy())  

    return np.vstack(features).astype(np.float32), np.array(labels).astype(np.int64)


# Extract features for training & validation
X_train, y_train = extract_features(resnet18, train_loader)
X_test, y_test = extract_features(resnet18, val_loader)

print("Feature extraction complete!")

Feature extraction complete!


In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Train SVM model
svm = SVC(kernel="linear")  # You can change "linear" to "rbf" for a nonlinear classifier
svm.fit(X_train, y_train)

# Make predictions on validation data
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ Model Training Complete!")
print(f"📈 SVM Accuracy on Validation Set: {accuracy * 100:.2f}%")

✅ Model Training Complete!
📈 SVM Accuracy on Validation Set: 98.69%


In [17]:
import pickle
import os

# Define the model save path
model_folder = r"C:/Users/abina/fish_recognition/models"
os.makedirs(model_folder, exist_ok=True)  # Ensure the folder exists

model_path = os.path.join(model_folder, "svm_model.pkl")

# Save the trained SVM model
with open(model_path, "wb") as f:
    pickle.dump(svm, f)

print(f"✅ SVM model saved at: {model_path}")

✅ SVM model saved at: C:/Users/abina/fish_recognition/models\svm_model.pkl


In [19]:
print("Model Exists:", os.path.exists(model_path))

Model Exists: True


In [25]:
fish_labels={0:"Bangus", 1:"Big Head Carp", 2:"Black Spotted Barb", 3:"Catfish", 4:"Climbing Perch", 5: "Fourfinger Threadfin", 6:"Freshwater Eel", 7:"Glass Perchlet",8:"Goby", 9:"Gold Fish", 10:"Gourami", 11:"Grass Carp", 12:"Green Spotted Puffer", 13:"Indian Carp", 14:"Indo-Pacific Tarpon", 15:"Jaguar Gapote", 16:"Janitor Fish", 17:"Knifefish", 18:"Long-Snouted Pipefish", 19:"Mosquito Fish", 20:"Mudfish", 21:"Mullet", 22:"Pangasius", 23:"Perch", 24:"Scat Fish", 25:"Silver Barb", 26:"Silver Carp", 27:"Silver Perch", 28:"Snakehead", 29:"Tenpounder", 30:"Tilapia"}

In [27]:
import numpy as np
import torch
import pickle
import torchvision.transforms as transforms
from PIL import Image

# Load the saved SVM model
with open(model_path, "rb") as f:
    svm_model = pickle.load(f)

# Load ResNet for feature extraction
resnet18 = torch.hub.load("pytorch/vision:v0.10.0", "resnet18", pretrained=True)
resnet18.fc = torch.nn.Identity()
resnet18.eval()

# Image transformation pipeline (matches Flask processing)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def extract_features(image):
    """Extract features from an image using ResNet18."""
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = resnet18(image).flatten().numpy()
    return features

# Test with n image
image_path = "C:/Users/abina/Downloads/Gourami 79.jpg" # Replace with actual image path
image = Image.open(image_path)

# Extract features and classify
features = extract_features(image)
prediction = svm_model.predict([features])[0]
fish_label = fish_labels.get(prediction, "Unknown Fish")

print(f"✅ Predicted fish category: {fish_label} ")


Using cache found in C:\Users\abina/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\abina\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abina\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Predicted fish category: Gourami 
